In [ ]:
import RMT

import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [ ]:
# torch.set_default_device('cuda')

num_inputs = 500
width = 1000
alpha = 2
sigma_W = 1.5
thetas = np.linspace(0, 2 * np.pi, num_inputs)
x0 = np.zeros((num_inputs, width), dtype=np.float32)
# A circle in the first two dimensions
x0[:, 0] = np.cos(thetas)
x0[:, 1] = np.sin(thetas)
fp_norm = (
    RMT.MFT_map(RMT.q_star_MC(alpha, sigma_W)[-1], alpha, sigma_W, usetqdm=False)[
        "postact_sq_mean"
    ][0]
    ** 0.5
)
x0 = x0 / np.mean(x0**2, axis=-1, keepdims=True)**0.5 * fp_norm
norm_dx0 = np.linalg.norm(np.diff(x0, axis=-2), axis=-1)
# display(f"{norm_dx0.mean():.2f} +- {norm_dx0.std():.2f}")
xs = np.array([x0, *RMT.MLP(x0, 250, alpha, sigma_W, fast=True)["postact"]])
# display(xs.shape)
norm_dxs = np.linalg.norm(np.diff(xs, axis=-2), axis=-1)
fig, ax = plt.subplots()
ax.plot(norm_dxs.mean(-1))
ax.fill_between(
    np.arange(norm_dxs.shape[0]),
    np.quantile(norm_dxs, 0.1, axis=-1),
    np.quantile(norm_dxs, 0.9, axis=-1),
    alpha=0.3,
)
ax.set(
    xlabel="layer",
    ylabel="norm_dxs",
)
fig, ax = plt.subplots()
ax.plot(norm_dxs.std(-1) / abs(norm_dxs.mean(-1)))
ax.set(
    xlabel="layer",
    ylabel="CV of norm_dxs",
)
fig, ax = plt.subplots()
ax.plot(np.log(norm_dxs).mean(-1))
ax.fill_between(
    np.arange(norm_dxs.shape[0]),
    np.quantile(np.log(norm_dxs), 0.1, axis=-1),
    np.quantile(np.log(norm_dxs), 0.9, axis=-1),
    alpha=0.3,
)
ax.set(
    xlabel="layer",
    ylabel="log norm_dxs",
)
fig, ax = plt.subplots()
ax.plot(np.log(norm_dxs).std(-1) / abs(np.log(norm_dxs).mean(-1)))
ax.set(
    xlabel="layer",
    ylabel="CV of log norm_dxs",
)
fig, ax = plt.subplots()
ax.plot(np.linalg.norm(xs, axis=-1).mean(-1))
ax.fill_between(
    np.arange(xs.shape[0]),
    np.quantile(np.linalg.norm(xs, axis=-1), 0.1, axis=-1),
    np.quantile(np.linalg.norm(xs, axis=-1), 0.9, axis=-1),
    alpha=0.3,
)
ax.set(
    xlabel="layer",
    ylabel="norm_xs",
)
# for i in range(norm_dxs.shape[0]):
#     print(f"Layer {i}: {norm_dxs[i].mean():.2f} +- {norm_dxs[i].std():.2f}")